In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################
import os
import secrets
from PIL import Image
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileAllowed
from wtforms import StringField, PasswordField, SubmitField, TextAreaField # BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms import StringField, PasswordField, SubmitField, TextAreaField
from wtforms.validators import InputRequired, Length, ValidationError # Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_security import SQLAlchemyUserDatastore
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta
import re

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    phone = db.Column(db.String(80))
    message = db.Column(db.String(80), unique=True)
    bio = db.Column(db.Text)
    admin = db.Column(db.Boolean)
    image_file = db.Column(db.String(20), nullable=False, default='default.jpg')
    notes = db.relationship('Note', backref='writer', lazy='dynamic')
    
class Note(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    author = db.Column(db.String(255))
    date_created = db.Column(db.TIMESTAMP, default=datetime.utcnow, nullable=False)

    
class Venues(db.Model):
    __tablename__ = 'venues'
    Name = db.Column(db.String(25), primary_key=True)
    address = db.Column(db.Float)
    score = db.Column(db.Float)
    url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    price = db.Column(db.Float)
    contact = db.Column(db.Float)
    location = db.Column(db.Text)
    time = db.Column(db.Text)
    review = db.Column(db.Float)
    
class ContactForm(FlaskForm):
    email = StringField("邮箱", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "邮箱"})
    username = StringField("姓名", validators=[InputRequired(), Length(min=1, max=15)], render_kw={"placeholder": "姓名"})
    phone = StringField("电话号码", validators=[InputRequired(), Length(min=6, max=13)], render_kw={"placeholder": "电话号码"})
    message = StringField("内容", widget=TextArea())
    submit = SubmitField("提交")


class RegisterForm(FlaskForm):
    email = StringField("邮箱", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "邮箱"})
    username = StringField("用户名", validators=[InputRequired(), Length(min=1, max=15)], render_kw={"placeholder": "用户名"})
    password = PasswordField("密码", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("注册")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("用户名已存在，请重新输入.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("邮箱已被注册，请重新输入.")


class LoginForm(FlaskForm):
    username = StringField("用户名", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "用户名"})
    password = PasswordField("密码", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "密码"})
    submit = SubmitField("登录")
                             
class ProfileForm(FlaskForm):
    username = StringField('用户名',
                           validators=[InputRequired(), Length(min=2, max=20)])
    email = StringField("邮箱", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "邮箱"})
    profile_body = StringField(u'Text', widget=TextArea())
    picture = FileField('更改头像', validators=[FileAllowed(['jpg', 'png'])])
    submit = SubmitField('提交')
    
class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "标题"})
    note_body = TextAreaField("Note Body", validators=[InputRequired(), Length(max=500)], render_kw={"placeholder":  "内容"})
    #note_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("发布到社区")

class UpdateNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "标题"})
    note_body = TextAreaField("Note Body", validators=[InputRequired(), Length(max=500)], render_kw={"placeholder":  "内容"})
    #note_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("发布")

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(text))                             
                             
@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index"))
    
        flash("用户不存在，或输入了无效的用户名/密码.")
    return render_template('login.html', title="登录", form=form)

@app.route('/lumao')
def lumao():
    return render_template('lumao.html')

@app.route('/lugou')
def lugou():
    return render_template('lugou.html')

@app.route('/zhaomao')
def zhaomao():
    return render_template('zhaomao.html')

@app.route('/gonglue')
def gonglue():
    return render_template('gonglue.html')

@app.route('/dog')
def dog():
    return render_template('dog.html')

@app.route('/cat')
def cat():
    return render_template('cat.html')

@app.route('/expert')
def expert():
    return render_template('expert.html')

@app.route('/rules', methods=['GET','POST'])
def rules():
    form = ContactForm()
    if form.validate_on_submit():
        Contact = User(username=form.username.data, email=form.email.data, phone=form.phone.data, message=form.message.data)
        db.session.add(Contact)
        db.session.commit()
        return redirect(url_for('index'))
    return render_template('rules.html', title='rules', form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password,bio='',admin=0)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='注册', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))
                             
@app.route('/new-note', methods=['GET','POST'])
@login_required
def new_note():
    form = NewNoteForm()
    if form.validate_on_submit():
        new_note = Note(title=form.title.data, note_body=form.note_body.data, writer=current_user, author=current_user.username)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('view_notes'))
    return render_template('new_note.html', title='New Note', form=form)

@app.route('/my-notes', methods=['GET','POST'])
@login_required
def view_notes():
    print('current user id:', current_user.id)
    if current_user.admin:
        # notes = Note.query.all()
        sqlalchemyObj = db.engine.execute('select note.id, note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id ORDER BY user.id')
    else:
        # dataset = Note.query.filter_by(writer=current_user).all()
        sql = 'select note.id, note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id WHERE note.note_writer == '+ str(current_user.id) + ' ORDER BY user.id'
        sqlalchemyObj = db.engine.execute(sql)
    notes = []
    for i in sqlalchemyObj:
        notes.append(i)
        # print(venues)
        dataset=[]
        dict={}
        for i in notes:
            dict['id'] = i[0]
            dict['title'] = i[1]
            dict['note_body'] = i[2]
            dict['date_created'] = i[3]
            dict['username'] = i[4]
            dict['image_file'] = i[5]
            # print(i)
            # print(dict)
            dataset.append(dict.copy())
    #print(dataset)
    return render_template('my_notes.html', notes=dataset, title='My Notes')


@app.route('/blogs', methods=['GET','POST'])
def blogs():
    sqlalchemyObj = db.engine.execute('select note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id ORDER BY user.id')
    notes = []
    for i in sqlalchemyObj:
        notes.append(i)
    # print(venues)
    dataset=[]
    dict={}
    for i in notes:
        dict['title'] = i[0]
        dict['note_body'] = i[1]
        dict['date_created'] = i[2]
        dict['username'] = i[3]
        dict['image_file'] = i[4]
        # print(i)
        # print(dict)
        dataset.append(dict.copy())
    #print(dataset)
    return render_template('blogs.html', notes=dataset, title='Blog Posts')

@app.route("/profile", methods=['GET', 'POST'])
@login_required
def profile():
    form = ProfileForm()
    if form.validate_on_submit():
        if form.picture.data:
            picture_file = save_picture(form.picture.data)
            current_user.image_file = picture_file
        current_user.username = form.username.data
        current_user.email = form.email.data
        current_user.bio = form.profile_body.data
        db.session.commit()
        flash('你的主页已更新!', 'success')
        return redirect(url_for('profile'))
    elif request.method == 'GET':
        form.username.data = current_user.username
        form.email.data = current_user.email
        form.profile_body.data = remove_html_tags(current_user.bio)
        if current_user.image_file is None:
            current_user.image_file = 'default.png'
    image_file = url_for('static', filename='profile_pics/' + current_user.image_file)
    return render_template('profile.html', title='Profile', image_file=image_file, form=form)

def save_picture(form_picture):
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_picture.filename)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(app.root_path, 'static/profile_pics', picture_fn)

    output_size = (125, 125)
    i = Image.open(form_picture)
    i.thumbnail(output_size)
    i.save(picture_path)

    return picture_fn
    

@app.route('/delete-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('view_notes'))

@app.route('/update-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def update_note(note_id):
    form = UpdateNoteForm()
    note = Note.query.get_or_404(note_id)
    if request.method == 'GET':
        form.title.data = note.title
        form.note_body.data = note.note_body
    if form.validate_on_submit():  # request.method="POST"
            note.query.filter_by(id=note_id).first()
            if note:
                note.title = form.title.data
                note.note_body = form.note_body.data
                db.session.commit()
                #print("Note id=",note_id)
                #print("Updated.")
                return redirect(url_for("view_notes"))

    return render_template('update_note.html', form=form, title='Update Note', id=note_id)


@app.route("/")
@app.route('/home')
def index():
    return render_template('index.html')


@app.route('/instruction')
def instruction():
    return render_template('instruction.html')


@app.route('/Contact', methods=['GET','POST'])
def Contact():
    form = ContactForm()
    if form.validate_on_submit():
        Contact = User(username=form.username.data, email=form.email.data, phone=form.phone.data, message=form.message.data)
        db.session.add(Contact)
        db.session.commit()
        return redirect(url_for('index'))
    return render_template('Contact.html', title='Contact', form=form)


@app.route("/Shop")
def Shop():

    sqlalchemyObj = db.engine.execute('select * from venues ORDER BY Name')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    # print(venues)
    dataset = []
    dict={}
    for i in venues:
        dict['Name'] = i[0]
        dict['Address'] = i[1]
        dict['Score'] = i[2]
        dict['Url'] = i[3]
        dict['Img_url'] = i[4]
        dict['Price'] = i[5]
        dict['Contact'] = i[6]
        dict['Location'] = i[6]
        dict['Time'] = i[7]
        dict['Review'] = i[8]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) #markers.append(fld.copy())


    return render_template('shop.html', title='Shop', dataset=dataset)

################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


################################################################################################################



################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 8892)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:8892/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 22:07:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:07:49] "GET /static/assets/img/bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:07:56] "GET /rules HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:28:48] "GET /instruction HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:28:48] "GET /static/assets/img/02.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:28:48] "GET /static/assets/img/1.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:28:48] "GET /static/assets/img/6.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 22:28:48] "GET /static/assets/img/5.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/D